In [1]:
!pip install transformers==4.44.2 torch==2.3.0 underthesea==6.8.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 75.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd
import numpy as np
import re
from underthesea import sent_tokenize

In [3]:
model_name = "wonrax/phobert-base-vietnamese-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()

tokenizer_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

bpe.codes: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/17.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [4]:
id2label = model.config.id2label
print("✅ Loaded model:", model_name)
print("Label mapping:", id2label)

✅ Loaded model: wonrax/phobert-base-vietnamese-sentiment
Label mapping: {0: 'NEG', 1: 'POS', 2: 'NEU'}


In [5]:
def get_phobert_sentiment(text):
    if not isinstance(text, str) or text.strip() == "":
        return "neutral", 0.0
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256,
        padding=True
    )
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        label_id = torch.argmax(probs, dim=1).item()
        label = id2label[label_id]
        score = probs[0, 2].item() - probs[0, 0].item()  # positive - negative

    return label, round(score, 3)

In [6]:
df = pd.read_csv("/kaggle/input/cogdis-data/Combined_Data.csv")
print(f"Loaded {len(df)} rows")

Loaded 20092 rows


In [7]:
results = df["Original Text"].apply(lambda x: pd.Series(get_phobert_sentiment(x)))
results.columns = ["sentiment_label", "sentiment_score"]
df = pd.concat([df, results], axis=1)

In [8]:
emotion_lexicon = {
    "buồn": ["buồn", "tuyệt vọng", "cô đơn", "chán", "khóc", "tổn thương"],
    "tức giận": ["ghét", "giận", "bực", "tức", "điên", "phẫn nộ", "ức chế"],
    "tội lỗi": ["hối hận", "tội lỗi", "xấu hổ", "ăn năn", "day dứt"],
    "sợ hãi": ["sợ", "hoảng", "run", "lo lắng", "ám ảnh", "kinh khủng"],
    "vui": ["vui", "hạnh phúc", "vui vẻ", "phấn khởi", "thoải mái"]
}

def detect_emotion_category(text):
    if not isinstance(text, str) or text.strip() == "":
        return "không xác định"
    counts = {emo: sum(1 for w in words if re.search(rf"\b{w}\b", text.lower())) 
              for emo, words in emotion_lexicon.items()}
    dominant = max(counts, key=counts.get)
    return dominant if counts[dominant] > 0 else "không xác định"

df["emotion_category"] = df["Original Text"].apply(detect_emotion_category)

In [9]:
strong_emotion_words = [
    "ghét", "kinh khủng", "vô vọng", "đáng sợ", "sợ", "điên rồ",
    "tồi tệ", "khủng khiếp", "đau khổ", "hoảng loạn", "tức giận", "chán nản"
]

def count_strong_emotion(text):
    if not isinstance(text, str): return 0
    return sum(len(re.findall(rf"\b{w}\b", text.lower())) for w in strong_emotion_words)

df["strong_emotion_count"] = df["Original Text"].apply(count_strong_emotion)

In [10]:
def emotion_variance(text):
    if not isinstance(text, str) or len(text.strip()) == 0:
        return 0.0
    sentences = sent_tokenize(text)
    if len(sentences) < 2:
        return 0.0
    scores = [get_phobert_sentiment(s)[1] for s in sentences]
    return round(np.var(scores), 3)

df["emotion_variance"] = df["Original Text"].apply(emotion_variance)

In [11]:
df.head(5)

,Original Text,Label,Emotional Reasoning,Overgeneralization,Should Statements,Personalization,Mental Filter,Disqualifying the Positive,Jumping to Conclusions,Labeling and Mislabeling,Magnification and Minimization,All-or-Nothing Thinking,sentiment_label,sentiment_score,emotion_category,strong_emotion_count,emotion_variance
0,"Cô ấy luôn có những hành động kỳ lạ, ví dụ như...",0,0,0,0,0,0,0,0,0,0,0,NEG,-0.936,tức giận,1,0.000
1,"Nếu bố mẹ tôi biết tôi đang vật lộn thế nào, h...",1,0,0,0,0,0,0,1,0,0,0,NEG,-0.966,không xác định,0,0.000
2,"Trong năm thứ 4 và thứ 5 của sự nghiệp, tôi kh...",1,0,0,0,0,0,0,0,0,0,1,NEG,-0.979,không xác định,0,0.001
3,"Tôi muốn bị ốm, và tôi biết điều đó thật khủng...",0,0,0,0,0,0,0,0,0,0,0,NEG,-0.604,không xác định,1,0.000
4,Trước khi chúng tôi bắt đầu hẹn hò và trong nă...,0,0,0,0,0,0,0,0,0,0,0,POS,0.100,không xác định,0,0.139


In [12]:
df.to_csv("Sentiment & Emotion Features.csv", index=False)